# Task #1: Get info Box (store in Python dictionary)

### Import necessary libaries

In [4]:
from bs4 import BeautifulSoup as bs
import requests

### Load the webpage

In [ ]:
req = requests.get(url='https://en.wikipedia.org/wiki/WALL-E')
# Convert to a beautiful soup object
soup = bs(req.content)

# Print out the HTML
contents = soup.prettify()
print(contents)

In [ ]:
info_box = soup.find(class_='infobox vevent')
info_rows = info_box.find_all('tr')
for row in info_rows:
    print(row.prettify())

In [ ]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', ' ') for li in row_data.find_all('li')]
    else:
        return row_data.get_text(' ', strip=True).replace('\xa0', ' ')

movie_info: dict = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find('th').get_text(' ', strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find('th').get_text(' ', strip=True)
        content_value = get_content_value(row.find('td'))
        movie_info[content_key] = content_value

movie_info

# Task #2: Get info box for all movies

In [34]:
r = requests.get(url='https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful soup object
soup = bs(r.content)

contents = soup.prettify()

In [ ]:
movies = soup.select('.wikitable.sortable i')
movies[0:10]
movies[0]

In [10]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', ' ') for li in row_data.find_all('li')]
    else:
        return row_data.get_text(' ', strip=True).replace('xa0', ' ')


def get_info_box(url: str) -> dict:


    req = requests.get(url=url)
    # Convert to a beautiful soup object
    soup = bs(req.content)

    info_box = soup.find(class_='infobox vevent')
    info_rows = info_box.find_all('tr')
    
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(' ', strip=True)
        elif index == 1:
            continue
        else:
            content_key = row.find('th').get_text(' ', strip=True)
            content_value = get_content_value(row.find('td'))
            movie_info[content_key] = content_value

    return movie_info

In [ ]:
%timeit
req = requests.get(url='https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful soup object
soup = bs(req.content)

base_path = 'https://en.wikipedia.org/'

movies = soup.select('.wikitable.sortable i a')
# movies[0:10]

movie_info_list: list = []
for index, movie in enumerate(movies):

    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']

        # В список добавляем словарь, который возвращает функция get_info_box
        movie_info_list.append(get_info_box(full_path))
    except Exception as e:
        print(movie.get_text())
        print(e)
        

In [ ]:
movie_info_list[0]

### Save/Reload Movie data

In [54]:
# Сохранить данные в JSON формат
import json
def save_data(title, data):
    with open(file=title, mode='w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
save_data('disney_data.json', movie_info_list)

In [14]:
# Загрузить данные из JSON формата
import json
def load_data(title):
    with open(title, mode='r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
load_data(title='disney_data.json')